In [ ]:
# pdf_path = "./data/UniversScience/dossiers-pédagogiques/marc/Flop chef.pdf"
# pdf_path = "./data/UniversScience/revue-decouverte/decouverte_436.pdf"
pdf_path = './data/UniversScience/revue-decouverte/decouverte_436.pdf'


In [ ]:
from pdf2image import convert_from_path
pages = convert_from_path(pdf_path)

In [ ]:
from ultralytics import YOLO
#https://huggingface.co/omoured/YOLOv10-Document-Layout-Analysis

model = YOLO('./models/yolov10x_best.pt').to('cuda:0')
# image = cv2.imread('input_sample.png')



In [ ]:
from PIL import Image
# Function to draw the bounding box and label on the image
def draw_bbox(image, x1, y1, x2, y2, label):
    import cv2
    import numpy as np
    
    # Convert the image to OpenCV format
    image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    
    # Draw the bounding box and label
    cv2.rectangle(image_cv, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(image_cv, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)
    
    # Convert the image back to PIL format
    image_pil = Image.fromarray(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB))
    
    return image_pil

for image in pages:
    result = model(image, conf=0.2, iou=0.8)[0]
    # Iterate through each detection
    for i, detection in enumerate(result.boxes):
        # Get the bounding box coordinates
        x1, y1, x2, y2 = detection.xyxy[0].cpu().numpy()
        
        # Get the confidence score
        confidence = detection.conf.item()
        
        # Get the class ID and name
        class_id = int(detection.cls)
        class_name = result.names[class_id]
        
        # Draw the bounding box and label on the image
        image = draw_bbox(image, x1, y1, x2, y2, class_name)

        # # Print the information
        # print(f"Detection {i+1}:")
        # print(f"  Class: {class_name}")
        # print(f"  Confidence: {confidence:.2f}")
        # print(f"  Bounding Box: ({x1:.2f}, {y1:.2f}) to ({x2:.2f}, {y2:.2f})")

        # Display the image with bounding box and label

    display(image)
